In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
# This is needed to display the images.
%matplotlib inline

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")

In [3]:
from utils import label_map_util

from utils import visualization_utils as vis_util

In [4]:
# What model to download.
MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11_17'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')

NUM_CLASSES = 90

In [5]:
opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
  file_name = os.path.basename(file.name)
  if 'frozen_inference_graph.pb' in file_name:
    tar_file.extract(file, os.getcwd())

In [6]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

In [7]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

W0902 18:56:29.434331  5056 deprecation_wrapper.py:119] From C:\Users\IVPL-D11\models\research\object_detection\utils\label_map_util.py:132: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.



In [8]:
from pykinect2 import PyKinectV2
from pykinect2.PyKinectV2 import *
from pykinect2 import PyKinectRuntime
from acquisitionKinect import AcquisitionKinect
from frame import Frame as Frame
import numpy as np
import cv2
import ctypes
import _ctypes
import sys
import face_recognition
import os
from scipy import io
import math
from gtts import gTTS
import pyttsx3
from textblob.classifiers import NaiveBayesClassifier
from textblob import TextBlob
engine = pyttsx3.init()
engine.setProperty('rate', 145)     # setting up new voice rate
engine.setProperty('volume',1.0)    # setting up volume level  between 0 and 1
voices = engine.getProperty('voices')       #getting details of current voice
engine.setProperty('voice', voices[0].id)  #changing index, changes voices.

#Load Basic Setting for Kinect
Kinect = AcquisitionKinect()
frame = Frame()

In [9]:
def loadingforAction():
    filename = 'KARDCADwithDrink3.mat'
    AM = io.loadmat(filename, mat_dtype = False)

    CiM_o = AM['Cim_out']

    CiM = np.array([])
    for i in range(25):
        CiM = np.concatenate((CiM,np.array(CiM_o)['value'][0,0][0][int(i)][0]))
    CiM = CiM.reshape((25, D))

    iMaxis_o = AM['IMaxis_out']
    iMaxis = np.array([])
    for i in range(3):
        iMaxis = np.concatenate((iMaxis,np.array(iMaxis_o)['value'][0,0][0][int(i)][0]))
    iMaxis = iMaxis.reshape((3, D))
    
    iMjoints_o = AM['IMjoints_out']
    iMjoints = np.array([])
    for i in range(14):
        iMjoints = np.concatenate((iMjoints,np.array(iMjoints_o)['value'][0,0][0][int(i)][0]))
    iMjoints = iMjoints.reshape((14, D))
    
    AM_o = AM['AM2str']
    AMM = np.array([])
    for i in range(4):
        AMM = np.concatenate((AMM,np.array(AM_o)['value'][0,0][0][int(i)][0]))
    AMM = AMM.reshape((4, D))
    
    return CiM, iMaxis, iMjoints, AMM



In [10]:
def cosAngle (u, v):
    cosAngle = np.dot(u,v)/(np.linalg.norm(u)*np.linalg.norm(v))
    return cosAngle


In [11]:
def binarizeHV (v):
    threshold = 0
    for i in range(len(v)):
        if v[i] > threshold:
            v[i] = 1
        else:
            v[i] = -1
    return v


In [12]:
def discretize(data, bins):
    split = np.array_split(np.sort(data), bins)
    cutoffs = [x[-1] for x in split]
    cutoffs = cutoffs[:-1]
    discrete = np.digitize(data, cutoffs, right=True)
    return discrete


In [13]:
def hdcResult(AM,M0,CiM,iMjoints,iMaxis,MAXL,compareAngle):
    #TP = np.zeros((1,17))
    #FN = np.zeros((1,17))
    #FP = np.zeros((1,17))
    predictedLabel = -1
    precision = 0
    recall = 0
    print("in")
    #for j in range (3):
    for i in range(1, 5):
        if(i == 1 or i == 2 or i==3 or i == 4 ):
            tmp = np.zeros(D)
            M = discretize(M0, MAXL)
            M = M.reshape((30, 42))
            for k in range( len(M)):
                    
                for n in range(len(M[0])):
                       
                    jointHDC = iMjoints[math.floor(n/3)]
                    axisHDC = iMaxis[n%3]
                     
                    valueHDC = CiM[M[k][n]]
                       #
                    tmp += ((jointHDC*axisHDC) * valueHDC)
                maxAngle = compareAngle
                for b in range (1,5):
                        
                    if(b == 1 or b == 2 or b==3 or b ==4 ):
                        angle = cosAngle (AM[b-1], binarizeHV(tmp))
                            
                        if (angle > maxAngle):
                            maxAngle = angle
                            predictedLabel = b
                    #if predictedLabel == i:
                        #TP(:,i) = TP(:,i)+1
                    #else:
                        #FN(:,i) = FN(:,i)+1;
                        #FP(:,predictedLabel) = FP(:,predictedLabel)+1
   #print("predictLabel:", predictedLabel)

    #precision = TP./(TP+FP)
    #recall = TP./(TP+FN)
    return predictedLabel


In [14]:

# setting for Face Recognition
known_face_encodings = []
known_face_names = []

# Load sample pictures and learn how to recognize it.
dirname = 'face_reco/knowns'
files = os.listdir(dirname)
for filename in files:
    name, ext = os.path.splitext(filename)
    if ext == '.jpg':
        known_face_names.append(name)
        pathname = os.path.join(dirname, filename)
        img = face_recognition.load_image_file(pathname)
        face_encoding = face_recognition.face_encodings(img)[0]
        known_face_encodings.append(face_encoding)

# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True



In [19]:
def make_sentence(depth, position, index, start, end):
    objects = ""
    objects += position + " "
    for i in range(start, end):
        objects += depth[i][0] + " and "

                                    
    objects += depth[end][0]

    return objects


# Functions for Action Recognition
D = 7000
numActivities = 3
MAXL = 25
compareAngle = 0.5
feature = np.zeros((20,3))
people = []
i = 1
framecheck = 0
max_depth = -1 
max_people = -1
frameset = np.array([])
norm_frameset = np.array([])
CiM, iMaxis, iMjoints, AMM = loadingforAction()
flag = 0
index = -1
framecheck = 0
#Object Detection
with detection_graph.as_default():
  with tf.Session(graph=detection_graph) as sess:
    while True:
        peop = 0
        situation=""
        # --- Getting frames and drawing
        Kinect.get_frame(frame)
        Kinect.get_color_frame()
        image_np = Kinect._kinect.get_last_color_frame()
        #image_np = Kinect._frameRGB
        image_depth = Kinect._frameDepthQuantized 
        Skeleton_img = Kinect._frameSkeleton
        image_np = np.reshape(image_np, (Kinect._kinect.color_frame_desc.Height, Kinect._kinect.color_frame_desc.Width, 4))
        image_np = cv2.cvtColor(image_np, cv2.COLOR_RGBA2RGB)
        show_img = image_np
       # show_img = image_np[ 200:1020, 350:1780]
        show_img = cv2.resize(show_img, (512,424))
        
        #image_np = cv2.cvtColor(image_np, cv2.COLOR_RGBA2RGB)
        #image_np = image_np[ 200:1020, 350:1780]
        #image_np = cv2.resize(image_np, (512,424))
        rgb_small_frame = cv2.resize(image_np, (0, 0), fx=0.25, fy=0.25)

# Face recognition
        if process_this_frame:
            # Find all the faces and face encodings in the current frame of video
            face_locations = face_recognition.face_locations(rgb_small_frame)
            face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

            face_names = []
            for face_encoding in face_encodings:
                distances = face_recognition.face_distance(known_face_encodings, face_encoding)
                min_value = min(distances)


                name = "Unknown person"
                if min_value < 0.3:
                    index = np.argmin(distances)
                    name = known_face_names[index]

                face_names.append(name)

        process_this_frame = not process_this_frame

       
        for (top, right, bottom, left), name in zip(face_locations, face_names):
            # Scale back up face locations since the frame we detected in was scaled to 1/4 size
            top *= 4
            right *= 4
            bottom *= 4
            left *= 4

            # Draw a box around the face
            cv2.rectangle(image_np, (left, top), (right, bottom), (0, 0, 255), 2)
            # Draw a label with a name below the face
            cv2.rectangle(image_np, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
            
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(image_np, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)
            
#---------------- Object Detection

        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')

        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')

        image_np_expanded = np.expand_dims(image_np, axis=0)
 
        (boxes, scores, classes, num) = sess.run(
            [detection_boxes, detection_scores, detection_classes, num_detections],
            feed_dict={image_tensor: image_np_expanded})

        vis_util.visualize_boxes_and_labels_on_image_array(
            image_np,
            np.squeeze(boxes),
            np.squeeze(classes).astype(np.int32),
            np.squeeze(scores),
            category_index,
            use_normalized_coordinates=True,
            line_thickness=8)


        coordinates = vis_util.return_coordinates(
                    image_np,
                    np.squeeze(boxes),
                    np.squeeze(classes).astype(np.int32),
                    np.squeeze(scores),
                    category_index,
                    use_normalized_coordinates=True,
                    line_thickness=8,
                    min_score_thresh=0.5)
        
        if coordinates is not None:
            depth_arr = [[0 for x in range(2)] for y in range(len(coordinates))]
            real_depth_arr = [[0 for x in range(2)] for y in range(len(coordinates))]
            count = 0
            for i in range(len(coordinates)):
                if coordinates[i][0] == 'person':
                    flag = 1
                    if count < len(face_names):
                        coordinates[i][0] = face_names[count]
                        count += 1
                        
            #print(coordinates)
            xpoint_for_depth=[]
            ypoint_for_depth=[]
            
            #print("object 개수:", len(coordinates))
            for i in range(len(coordinates)):
                ypoint_for_depth.append(int((coordinates[i][1]+coordinates[i][2])/2))
                xpoint_for_depth.append(int((coordinates[i][3]+coordinates[i][4])/2))

            #print("ypoint_for_depth:", ypoint_for_depth)
            #print("xpoint_for_depth:", xpoint_for_depth)
            m = 0
            for i in range(len(coordinates)):
                if(coordinates[i][0]!="N/A" and coordinates[i][0] not in depth_arr):
                    depth_arr[m][0] = coordinates[i][0]
                    depth_arr[m][1] = 0
                    for x in range (-5, 6):
                        for y in range(-5, 6):
                            depth_arr[m][1] += image_depth[int((ypoint_for_depth[i]+x) * 424 / 1080)][int((xpoint_for_depth[i]+y) * 512 / 1920)]
                            show_img[int((ypoint_for_depth[i]+x) * 424 / 1080)][int((xpoint_for_depth[i]+y) * 512 / 1920)] = [255, 0,95]
                    depth_arr[m][1] /= 121
                    m+=1

            # Compare each objects' depth. (Sorting: close -> far)
            depth_arr.sort(key=lambda x:x[1])
            print(depth_arr)
       
               # print(coordinates[i][0], "'s depth: ", Pixel_Depth)
        image_np = cv2.resize(image_np,  (640, 480))
        cv2.imshow("ee", image_np)  
        
        cv2.imshow("depthh", image_depth)
        #cv2.imshow('KINECT Color', image_np)
        
        # Rule of Generating Sentence
        if(len(face_names)> max_people):
            print("face: ", face_names)
            for i in range(len(face_names)): 
                if face_names[i] not in people:
                    people.append( face_names[i])
            print("people: ", people)
            print("people_max: ", max_people)
            max_people = len(people)
            
        if(len(depth_arr)> max_depth):
            tmp_situation = ""
            for i in range(len(depth_arr)):
                if(depth_arr[i][0] in face_names or depth_arr[i][0]== "person"):
                    index = i
            print("index: ", index)

            print("detected object: \n", depth_arr)                        
            if index==0:
                tmp_situation += make_sentence(depth_arr, "in front of", index, 1, len(depth_arr)-1 )
            elif index == len(depth_arr)-1:
                tmp_situation += make_sentence(depth_arr, "behind", index, 0, len(depth_arr)-2)
            elif index > 0 and index <len(depth_arr)-1 :
                tmp_situation += make_sentence(depth_arr, "behind", index, 0, index-1 )
                tmp_situation += " and "
                tmp_situation += make_sentence(depth_arr, "in front of", index, index + 1, len(depth_arr)-1 )
            max_depth = len(depth_arr)
#Action Recognition
       
        
        if Skeleton_img is not None:

            
            cv2.resize(Skeleton_img, (960, 540))
            cv2.imshow("joint", Skeleton_img)
            print("find", framecheck)            
           
            feature[1][0] = Kinect.joint_points3D[3].x
            feature[1][1] = Kinect.joint_points3D[3].y
            feature[1][2] = Kinect.joint_points3D[3].z
            
            feature[2][0] = Kinect.joint_points3D[2].x
            feature[2][1] = Kinect.joint_points3D[2].y
            feature[2][2] = Kinect.joint_points3D[2].z
            
            feature[3][0] = Kinect.joint_points3D[8].x
            feature[3][1] = Kinect.joint_points3D[8].y
            feature[3][2] = Kinect.joint_points3D[8].z

            feature[4][0] = Kinect.joint_points3D[9].x
            feature[4][1] = Kinect.joint_points3D[9].y
            feature[4][2] = Kinect.joint_points3D[9].z

            feature[5][0] = Kinect.joint_points3D[11].x
            feature[5][1] = Kinect.joint_points3D[11].y
            feature[5][2] = Kinect.joint_points3D[11].z

            feature[6][0] = Kinect.joint_points3D[4].x
            feature[6][1] = Kinect.joint_points3D[4].y
            feature[6][2] = Kinect.joint_points3D[4].z

            feature[7][0] = Kinect.joint_points3D[5].x
            feature[7][1] = Kinect.joint_points3D[5].y
            feature[7][2] = Kinect.joint_points3D[5].z

            feature[8][0] = Kinect.joint_points3D[7].x
            feature[8][1] = Kinect.joint_points3D[7].y
            feature[8][2] = Kinect.joint_points3D[7].z

            feature[9][0] = Kinect.joint_points3D[1].x
            feature[9][1] = Kinect.joint_points3D[1].y
            feature[9][2] = Kinect.joint_points3D[1].z

            feature[10][0] = Kinect.joint_points3D[16].x
            feature[10][1] = Kinect.joint_points3D[16].y
            feature[10][2] = Kinect.joint_points3D[16].z

            feature[11][0] = Kinect.joint_points3D[17].x
            feature[11][1] = Kinect.joint_points3D[17].y
            feature[11][2] = Kinect.joint_points3D[17].z

            feature[12][0] = Kinect.joint_points3D[19].x
            feature[12][1] = Kinect.joint_points3D[19].y
            feature[12][2] = Kinect.joint_points3D[19].z

            feature[13][0] = Kinect.joint_points3D[12].x
            feature[13][1] = Kinect.joint_points3D[12].y
            feature[13][2] = Kinect.joint_points3D[12].z

            feature[14][0] = Kinect.joint_points3D[13].x
            feature[14][1] = Kinect.joint_points3D[13].y
            feature[14][2] = Kinect.joint_points3D[13].z

            feature[15][0] = Kinect.joint_points3D[15].x
            feature[15][1] = Kinect.joint_points3D[15].y
            feature[15][2] = Kinect.joint_points3D[15].z
            tmp = []
            for i in range (1, 16):
                for j in range(3):
                    tmp.append(feature[i][j])

            frameset = np.concatenate((frameset, tmp), axis = 0)
            framecheck+=1
            if(framecheck == 30):
                frameset = frameset.reshape((framecheck, 45))
                for row in range(len(frameset)):
                    
                    torsox = float(frameset[row][24])
                    torsoy = float(frameset[row][25])
                    torsoz = float(frameset[row][26])
                    neckx = float(frameset[row][3])
                    necky = float(frameset[row][4])
                    neckz = float(frameset[row][5])
                    denom = math.sqrt(math.pow((neckx - torsox),2) + math.pow((necky - torsoy),2) + math.pow((neckz - torsoz),2))
                    tmp = []
                    i = 0
                    while (i < 24):
                        tmp.append((float(frameset[row][i]) - torsox)/denom)
                        tmp.append((float(frameset[row][i+1]) - torsoy)/denom)
                        tmp.append((float(frameset[row][i+2]) - torsoz)/denom)
                        i+=3
                    i = 27
                    while (i < 45):
                        tmp.append((float(frameset[row][i]) - torsox)/denom)
                        tmp.append((float(frameset[row][i+1]) - torsoy)/denom)
                        tmp.append((float(frameset[row][i+2]) - torsoz)/denom)
                        i+=3
                    norm_frameset = np.concatenate((norm_frameset, tmp), axis = 0)
                    
                #norm_frameset = norm_frameset.reshape((k, 42))
               # CiM,iMjoints,iMaxis = initItemMemories (D, MAXL)
                
                predictedLabel = hdcResult(AMM,norm_frameset,CiM,iMjoints,iMaxis,MAXL,compareAngle)
                print("Action Detect----------------------------------------------------------")
                if(predictedLabel == 1):
                    action = "standing"
                    print("still")
                elif (predictedLabel == 2 ):
                    action = "sitting"
                    print ("sit")
                elif (predictedLabel == 3 ):
                    action = "walking"
                    print ("walk")
                elif predictedLabel == 4 :
                    action = "drinking water"
                    print("drink")
                print("-----------------------------------------------------------------------")
               
                tmp = []
                frameset = np.array([])
                norm_frameset = np.array([])
                framecheck = 0
                
                            
       
                back_objects = ""
                front_objects = ""
                if(flag==1):
                    if(max_people <2): # one person
                        if(max_people ==1):
                            situation += people[0] + " is "
                        elif max_people <1:
                            situation += "Unknown person is "
                        situation += action + " "
                        situation += tmp_situation
                        #back_objects += situation
                        #front_objects += situation
                        
                        
                    elif (max_people>=2) : # more than two people
                        for i in range(len(people)-1):
                            situation += people[i] + " and "
                        situation += people[len(people)-1]+ " are near by "
                   # if "person" in Pixel_Depth_Dict.keys():

                        situation += action
                    situation += "."
                    print("eng: ", situation)
                    word = TextBlob(situation)
                    fin_situation = str(word.translate(from_lang='en', to='ko'))
                    engine.say(fin_situation)
                    engine.runAndWait()
                    print("ko: ", fin_situation)
                    max_depth = -1
                    max_people = -1
                    people = []
                    #tts = gTTS(text=fin_situation, lang='ko')
                    #tts.save("helloKO.mp3")                    
        i=i+1
       # print("--------------------------------------------------- people: ", people)
       # print("--------------------------------------------------- situation: ", situation)
       # print("--------------------------------------------------- face: ", face_names)
        
                
        

        cv2.waitKey(1)


        key = cv2.waitKey(1)
        if key == 27: break

[['person', 165.12396694214877], ['couch', 172.98347107438016]]
face:  []
people:  []
people_max:  -1
index:  0
detected object: 
 [['person', 165.12396694214877], ['couch', 172.98347107438016]]
find 0
[['person', 168.73553719008265]]
find 1
[['person', 166.8595041322314]]
find 2
[['person', 167.69421487603304]]
find 3
[['person', 168.13223140495867]]
find 4
[['person', 168.2396694214876]]
find 5
[['person', 168.47107438016528]]
find 6
[['person', 167.87603305785123]]
find 7
[['Unknown person', 167.69421487603304]]
face:  ['Unknown person']
people:  ['Unknown person']
people_max:  0
find 8
[['Unknown person', 167.70247933884298]]
find 9
[['Unknown person', 168.40495867768595]]
find 10
[['Unknown person', 166.54545454545453]]
find 11
[['Unknown person', 168.89256198347107]]
find 12
[['Unknown person', 167.1487603305785]]
find 13
[['Unknown person', 169.30578512396696]]
find 14
[['Unknown person', 166.25619834710744]]
find 15
[['Unknown person', 167.61157024793388]]
find 16
[['Unknown pe

[['Unknown person', 151.0082644628099]]
face:  ['Unknown person']
people:  ['Unknown person']
people_max:  0
find 14
[['Unknown person', 146.3388429752066]]
find 15
[['Unknown person', 126.6694214876033], ['couch', 181.22314049586777]]
find 16
[['Unknown person', 112.79338842975207]]
find 17
[['couch', 117.91735537190083]]
find 18
[['Unknown person', 94.04132231404958]]
find 19
[['Unknown person', 86.39669421487604]]
find 20
[['Unknown person', 50.54545454545455]]
find 21
[['person', 42.3801652892562]]
find 22
[['person', 24.884297520661157]]
find 23
[['person', 19.0]]
find 24
[['person', 27.0]]
find 25
[['person', 40.96694214876033]]
find 26
[]
find 27
[]
find 28
[]
find 29
in
Action Detect----------------------------------------------------------
sit
-----------------------------------------------------------------------
eng:  Unknown person is sitting in front of couch.
ko:  알 수없는 사람이 소파 앞에 앉아있다.
[['person', 18.768595041322314]]
face:  []
people:  []
people_max:  -1
index:  0
detect

KeyboardInterrupt: 